In [273]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from pandas_profiling import ProfileReport


In [274]:
test=pd.read_csv("test.csv")

In [275]:
test.shape

(418, 11)

In [276]:
train=pd.read_csv("train.csv")

In [277]:
train.shape

(891, 12)

In [278]:
train.duplicated().sum()


0

In [279]:
train["Sex"].replace({"male":0,"female":1},inplace=True)

In [280]:
df=pd.concat([train,test],axis=0)

In [281]:
df["Embarked"].fillna(("S"),inplace=True)

In [282]:
df.corr()

,Age,Fare,Parch,PassengerId,Pclass,SibSp,Survived
Age,1.000000,0.178740,-0.150917,0.028814,-0.408106,-0.243699,-0.077221
Fare,0.178740,1.000000,0.221539,0.031428,-0.558629,0.160238,0.257307
Parch,-0.150917,0.221539,1.000000,0.008942,0.018322,0.373587,0.081629
PassengerId,0.028814,0.031428,0.008942,1.000000,-0.038354,-0.055224,-0.005007
Pclass,-0.408106,-0.558629,0.018322,-0.038354,1.000000,0.060832,-0.338481
SibSp,-0.243699,0.160238,0.373587,-0.055224,0.060832,1.000000,-0.035322
Survived,-0.077221,0.257307,0.081629,-0.005007,-0.338481,-0.035322,1.000000


In [283]:
df["Sex"].replace({"male":0,"female":1},inplace=True)

In [284]:
df["Sex"]

0      0
1      1
2      1
3      1
4      0
5      0
6      0
7      0
8      1
9      1
10     1
11     1
12     0
13     0
14     1
15     1
16     0
17     0
18     1
19     1
20     0
21     0
22     1
23     0
24     1
25     1
26     0
27     0
28     1
29     0
      ..
388    0
389    0
390    0
391    1
392    0
393    0
394    0
395    1
396    0
397    1
398    0
399    0
400    1
401    0
402    1
403    0
404    0
405    0
406    0
407    0
408    1
409    1
410    1
411    1
412    1
413    0
414    1
415    0
416    0
417    0
Name: Sex, Length: 1309, dtype: int64

In [285]:
df["Age"].fillna((df["Age"].median()),inplace=True)

In [286]:
df["Age"].isna().sum()

0

In [287]:
df["Fare"] = df.groupby("Pclass").transform(lambda x: x.fillna(x.mean()))

In [288]:
df["Fare"].isna().sum()

0

In [289]:
df.columns


Index(['Age', 'Cabin', 'Embarked', 'Fare', 'Name', 'Parch', 'PassengerId',
       'Pclass', 'Sex', 'SibSp', 'Survived', 'Ticket'],
      dtype='object')

In [290]:
df.drop(columns=["Cabin","Name","PassengerId","Ticket"],inplace=True)

In [291]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 8 columns):
Age         1309 non-null float64
Embarked    1309 non-null object
Fare        1309 non-null float64
Parch       1309 non-null int64
Pclass      1309 non-null int64
Sex         1309 non-null int64
SibSp       1309 non-null int64
Survived    891 non-null float64
dtypes: float64(3), int64(4), object(1)
memory usage: 92.0+ KB


In [292]:
cat_column = df.select_dtypes(exclude=np.number).columns

In [293]:
num_column = df.select_dtypes(include=np.number).columns

In [294]:
cat_col=pd.get_dummies(df[cat_column])

In [295]:
cat_col


,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
5,0,1,0
6,0,0,1
7,0,0,1
8,0,0,1
9,1,0,0


In [296]:
num_column


Index(['Age', 'Fare', 'Parch', 'Pclass', 'Sex', 'SibSp', 'Survived'], dtype='object')

In [297]:
df_preprocessed=pd.concat([df[num_column],cat_col],axis=1)


In [298]:
df_preprocessed

,Age,Fare,Parch,Pclass,Sex,SibSp,Survived,Embarked_C,Embarked_Q,Embarked_S
0,22.0,22.0,0,3,0,1,0.0,0,0,1
1,38.0,38.0,0,1,1,1,1.0,1,0,0
2,26.0,26.0,0,3,1,0,1.0,0,0,1
3,35.0,35.0,0,1,1,1,1.0,0,0,1
4,35.0,35.0,0,3,0,0,0.0,0,0,1
5,28.0,28.0,0,3,0,0,0.0,0,1,0
6,54.0,54.0,0,1,0,0,0.0,0,0,1
7,2.0,2.0,1,3,0,3,0.0,0,0,1
8,27.0,27.0,2,3,1,0,1.0,0,0,1
9,14.0,14.0,0,2,1,1,1.0,1,0,0


In [299]:
test_processed = df_preprocessed[df_preprocessed["Survived"].isna()]

In [300]:
train_preprocessed=df_preprocessed[df_preprocessed["Survived"].notna()]

In [301]:
test_X=test_processed.drop(columns=["Survived"])

In [302]:
test_X

,Age,Fare,Parch,Pclass,Sex,SibSp,Embarked_C,Embarked_Q,Embarked_S
0,34.5,34.5,0,3,0,0,0,1,0
1,47.0,47.0,0,3,1,1,0,0,1
2,62.0,62.0,0,2,0,0,0,1,0
3,27.0,27.0,0,3,0,0,0,0,1
4,22.0,22.0,1,3,1,1,0,0,1
5,14.0,14.0,0,3,0,0,0,0,1
6,30.0,30.0,0,3,1,0,0,1,0
7,26.0,26.0,1,2,0,1,0,0,1
8,18.0,18.0,0,3,1,0,1,0,0
9,21.0,21.0,0,3,0,2,0,0,1


In [303]:
train_X=train_preprocessed.drop(columns=["Survived"])

In [304]:
train_X["Age"]

0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
5      28.0
6      54.0
7       2.0
8      27.0
9      14.0
10      4.0
11     58.0
12     20.0
13     39.0
14     14.0
15     55.0
16      2.0
17     28.0
18     31.0
19     28.0
20     35.0
21     34.0
22     15.0
23     28.0
24      8.0
25     38.0
26     28.0
27     19.0
28     28.0
29     28.0
       ... 
861    21.0
862    48.0
863    28.0
864    24.0
865    42.0
866    27.0
867    31.0
868    28.0
869     4.0
870    26.0
871    47.0
872    33.0
873    47.0
874    28.0
875    15.0
876    20.0
877    19.0
878    28.0
879    56.0
880    25.0
881    33.0
882    22.0
883    28.0
884    25.0
885    39.0
886    27.0
887    19.0
888    28.0
889    26.0
890    32.0
Name: Age, Length: 891, dtype: float64

In [305]:
train_y=train_preprocessed["Survived"]

In [306]:
train_y.shape

(891,)

In [307]:
test_y=test_processed["Survived"]

In [308]:
test_y.shape

(418,)

# Logistic Regression

In [309]:
from sklearn.linear_model import LogisticRegression

In [310]:
log=LogisticRegression()

In [311]:
log.fit(train_X,train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [312]:
train_y_pred=log.predict(train_X)

In [313]:
from sklearn.metrics import accuracy_score
print("Accuracy score train:",accuracy_score(train_y,train_y_pred))

Accuracy score train: 0.8013468013468014


In [314]:
test_y_pred=log.predict(test_X)

In [315]:
test_y_pred

array([0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
       0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
       1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0.,
       0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0.,
       1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1.,
       1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1.,
       0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 1.

In [316]:
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, classification_report

In [317]:
recall_score(train_y,train_y_pred)

0.7017543859649122

In [318]:
test_y_pred=pd.DataFrame(test_y_pred,columns= ["Survived"])

In [319]:
log_result = pd.concat([test[["PassengerId"]],test_y_pred] ,axis = 1)

In [320]:
log_result

,PassengerId,Survived
0,892,0.0
1,893,0.0
2,894,0.0
3,895,0.0
4,896,1.0
5,897,0.0
6,898,1.0
7,899,0.0
8,900,1.0
9,901,0.0


In [321]:
log_result.to_csv("log.csv",index = False)

# KNN Classifier

In [322]:
from sklearn.neighbors import KNeighborsClassifier

In [369]:
knn=KNeighborsClassifier(n_neighbors=3)

In [370]:
knn.fit(train_X,train_y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [372]:
ktrain_y_pred=knn.predict(train_X)

In [373]:
accuracy_score(train_y,ktrain_y_pred)

0.8641975308641975

In [374]:
ktest_y_pred=knn.predict(test_X)

In [375]:
recall_score(train_y,ktrain_y_pred)

0.7339181286549707

In [376]:
ktest_y_pred=pd.DataFrame(ktest_y_pred,columns= ["Survived"])

In [377]:
knn_result = pd.concat([test[["PassengerId"]],ktest_y_pred] ,axis = 1)

In [378]:
knn_result.to_csv("knn.csv",index = False)

# Ridge Regression 

In [379]:
from sklearn.model_selection import train_test_split


In [383]:
X_train,X_test,y_train,y_test=train_test_split(train_X,train_y,test_size=0.3,random_state=10)

In [390]:
rd=LogisticRegression(penalty="l2")

In [391]:
rd.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [392]:
rdtrain_pred=rd.predict(X_train)

In [399]:
recall_score(y_train,rdtrain_pred)

0.7016129032258065

In [394]:
rdtest_pred=rd.predict(X_test)

In [398]:
recall_score(y_test,rdtest_pred)

0.7021276595744681

In [400]:
rdtest_y_pred=rd.predict(test_X)


In [401]:
rdtest_y_pred=pd.DataFrame(rdtest_y_pred,columns= ["Survived"])

In [402]:
ridge_result = pd.concat([test[["PassengerId"]],rdtest_y_pred] ,axis = 1)

In [403]:
ridge_result.to_csv("Ridge_result.csv",index=False)

In [404]:
train_X.columns

Index(['Age', 'Fare', 'Parch', 'Pclass', 'Sex', 'SibSp', 'Embarked_C',
       'Embarked_Q', 'Embarked_S'],
      dtype='object')

# Naive Bayes Classifier

In [407]:
from sklearn.naive_bayes import GaussianNB

In [408]:
gnb=GaussianNB()

In [410]:
X_train.drop(columns=['Embarked_C','Embarked_Q','Embarked_S'],inplace=True)

In [412]:
X_test.drop(columns=['Embarked_C','Embarked_Q','Embarked_S'],inplace=True)

In [413]:
test_X.drop(columns=['Embarked_C','Embarked_Q','Embarked_S'],inplace=True)

In [414]:
gnb.fit(X_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [415]:
gnbtrain_pred=gnb.predict(X_train)

In [416]:
accuracy_score(y_train,gnbtrain_pred)

0.7672552166934189

In [419]:
gnbtest_pred=gnb.predict(X_test)

In [420]:
accuracy_score(y_test,gnbtest_pred)

0.7574626865671642

In [421]:
recall_score(y_test,gnbtest_pred)

0.7659574468085106

In [422]:
gnbtest_y_pred=gnb.predict(test_X)

In [423]:
gnbtest_y_pred=pd.DataFrame(gnbtest_y_pred,columns= ["Survived"])

In [424]:
naive=pd.concat([test["PassengerId"],gnbtest_y_pred] ,axis = 1)

In [426]:
naive.to_csv("GaussianNb.csv",index=False)